<a href="https://colab.research.google.com/github/norhan333/NeuroPhone-RealTime-BrainMobilePhone-Interface/blob/main/model/EPFL_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!!pip install -q MOABB

[]

In [1]:
import numpy as np
import scipy.signal
import scipy.io as sio
import matplotlib
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

In [2]:
import moabb
from moabb.datasets import EPFLP300, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance
from mne.filter import construct_iir_filter
moabb.set_log_level("info")


In [3]:
def getData(subjectNumbers,dataset=EPFLP300()):
  return dataset.get_data(subjectNumbers)

In [4]:
def getSubjectData(subjectNumber,session_number,run_number):
  session = f'session_{session_number}'
  run = f'run_{run_number}'
  return getData([subjectNumber])[subjectNumber][session][run]

In [5]:
def filterData(data,lp=12,hp=1,sf=2048,resample=32):
  iir_params = dict(order=4, ftype='butter', output='sos')  
  iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
  data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
  return data

In [ ]:
lp=12;hp=1;sf=2048;resample=32
iir_params = dict(order=4, ftype='butter', output='sos')  
iir_params = construct_iir_filter(iir_params, [hp,lp], None, sf, 'bandpass', return_copy=False)
data=getSubjectData(1,1,1)
data
# x=data.filter(method='iir',iir_params=iir_params,h_freq=hp,l_freq=lp)
# x

In [16]:
subjects={}
for subjectNumber in range(1,9):
  sessions={}
  for sessionNumber in range(1,5):
    runs={}
    for runNumber in range(1,7):
      data = getSubjectData(subjectNumber,sessionNumber,runNumber)
      data = filterData(data)
      runs[runNumber]=data.resample(32)
      #todo normalization
    sessions[sessionNumber]=runs
  subjects[subjectNumber]=sessions

In [ ]:
subjects[1][1][1]

In [11]:
def bandPassFilter(data, sampligFrequency, lowFrequency, highFrequency, order):
  sos=scipy.signal.butter(order, Wn=[lowFrequency,highFrequency],btype='bandpass', fs=sampligFrequency, output='sos')
  filteredData = scipy.signal.sosfilt(sos, data)
 #plotting
  t = np.linspace(0, 1, 114687 , False)  # 1 second
  fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
  ax1.plot(t, filteredData)
  ax1.set_title('10 Hz and 20 Hz sinusoids')
  ax1.axis([0, 1,1, 20])

  ax2.plot(t, filteredData)
  ax2.set_title('After 15 Hz high-pass filter')
  ax2.axis([0, 1,1, 20])
  ax2.set_xlabel('Time [seconds]')
  plt.tight_layout()
  plt.show()
  return filteredData